# IMportaciones

In [1]:
import os
import sys
import torch
import pandas as pd
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(project_root)

if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils import load_config, get_device, set_seed
from src.data import CharTokenizer
from src.models.transformer import MicroTransformer
from src.decoding import (
    decode_greedy,
    decode_beam_search,
    decode_topk_topp,
)


# Funciones

In [2]:

CONFIG_PATH = "configs/train.yaml"
cfg = load_config(CONFIG_PATH)

set_seed(cfg["train"].get("seed", 42))
device = get_device(cfg["train"].get("device", "cuda"))
print("Device:", device)


tokenizer = CharTokenizer.load("data/vocab.json")
vocab_size = tokenizer.vocab_size
print("Vocab_size:", vocab_size)

model_cfg = cfg["model"]

def load_trained_model(posenc_type: str, checkpoint: str):

    model = MicroTransformer(
        vocab_size=vocab_size,
        d_model=model_cfg["d_model"],
        n_heads=model_cfg["n_heads"],
        n_layers=model_cfg["n_layers"],
        d_ff=model_cfg["d_ff"],
        dropout=model_cfg.get("dropout", 0.1),
        max_seq_len=model_cfg["max_seq_len"],
        posenc_type=posenc_type,
    ).to(device)

    state = torch.load(checkpoint, map_location=device)


    if isinstance(state, dict) and "model_state" in state:
        model.load_state_dict(state["model_state"])
    elif isinstance(state, dict) and "model_state_dict" in state:
        model.load_state_dict(state["model_state_dict"])
    else:
        model.load_state_dict(state)

    model.eval()
    print(f"Modelo cargado ({posenc_type}) desde {checkpoint}")
    return model

def generate_samples(model, strategy: str, prompts, max_new_tokens=200, **kwargs):

    rows = []
    for prompt in prompts:
        if strategy == "greedy":
            text = decode_greedy(
                model,
                tokenizer,
                prompt,
                max_new_tokens=max_new_tokens,
                device=device,
            )
        elif strategy == "beam":
            text = decode_beam_search(
                model,
                tokenizer,
                prompt,
                max_new_tokens=max_new_tokens,
                device=device,
                **kwargs,
            )
        elif strategy == "topk_topp":
            text = decode_topk_topp(
                model,
                tokenizer,
                prompt,
                max_new_tokens=max_new_tokens,
                device=device,
                **kwargs,
            )
        else:
            raise ValueError(f"Estrategia desconocida: {strategy}")

        rows.append(
            {
                "strategy": strategy,
                "prompt": prompt,
                "sample": text,
            }
        )

    df = pd.DataFrame(rows)
    display(df)
    return df


[2025-11-22 09:10:52,403] INFO - Seed fijada a 42
[2025-11-22 09:10:52,428] INFO - Vocabulario cargado con 65 caracteres.


Device: cuda
Vocab_size: 65


# Ejecuciones

In [26]:

posenc_type_main = "rope"                     
checkpoint_main = "checkpoints/best_rope.pt"       

model_main = load_trained_model(posenc_type_main, checkpoint_main)


Modelo cargado (rope) desde checkpoints/best_rope.pt


C:\Users\USER\AppData\Local\Temp\ipykernel_11412\1931491145.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(checkpoint, map_location=device)


In [27]:

prompts = [
    "ROMEO:",
    "JULIET:",
    "KING:",
    "MACBETH:",
]

prompts


['ROMEO:', 'JULIET:', 'KING:', 'MACBETH:']

In [28]:


os.makedirs("samples", exist_ok=True)

df_greedy = generate_samples(
    model_main,
    strategy="greedy",
    prompts=prompts,
    max_new_tokens=200,
)

df_greedy.to_csv("samples/greedy_samples.csv", index=False)
print("Muestras greedy guardadas en samples/greedy_samples.csv")


,strategy,prompt,sample
0,greedy,ROMEO:,ROMEO:\nWhat he will the son the shall the sha...
1,greedy,JULIET:,JULIET:\nThe so the shall the son the shall th...
2,greedy,KING:,KING:\nI will the shall the son the shall the ...
3,greedy,MACBETH:,MACBETH:\nThe so the shall the son the shall t...


Muestras greedy guardadas en samples/greedy_samples.csv


In [29]:

df_beam = generate_samples(
    model_main,
    strategy="beam",
    prompts=prompts,
    max_new_tokens=200,
    beam_size=4,
    length_penalty=0.7,
)

df_beam.to_csv("samples/beam_samples.csv", index=False)
print("Muestras beam guardadas en samples/beam_samples.csv")


,strategy,prompt,sample
0,beam,ROMEO:,ROMEO:\nWhat you have that with the will the s...
1,beam,JULIET:,JULIET:\nWhat you have to the will of the word...
2,beam,KING:,KING:\nWhat you have that with the will the wi...
3,beam,MACBETH:,MACBETH:\nThat you have heart the will the wil...


Muestras beam guardadas en samples/beam_samples.csv


In [30]:

df_topp = generate_samples(
    model_main,
    strategy="topk_topp",
    prompts=prompts,
    max_new_tokens=200,
    top_k=50,
    top_p=0.9,
    temperature=0.8,
)

df_topp.to_csv("samples/topk_topp_samples.csv", index=False)
print("Muestras top-k/top-p guardadas en samples/topk_topp_samples.csv")


,strategy,prompt,sample
0,topk_topp,ROMEO:,"ROMEO:\nNo, then he with his love and of the w..."
1,topk_topp,JULIET:,JULIET:\nBe the mares you more the sea is peac...
2,topk_topp,KING:,"KING:\nThe how he arm with by forth, with the ..."
3,topk_topp,MACBETH:,MACBETH:\nThe for bear the such the pring of t...


Muestras top-k/top-p guardadas en samples/topk_topp_samples.csv


In [8]:


dfs = []
for name in ["greedy", "beam", "topk_topp"]:
    path = f"samples/{name}_samples.csv"
    if os.path.exists(path):
        df = pd.read_csv(path)
        dfs.append(df)

if dfs:
    all_samples = pd.concat(dfs, ignore_index=True)
    display(all_samples)
else:
    print("No hay muestras guardadas todavía.")


,strategy,prompt,sample
0,greedy,ROMEO:,ROMEO:\nWhat should be so the send of the cons...
1,greedy,JULIET:,JULIET:\nWhat should not so stand to the senat...
2,greedy,KING:,KING:\nI will not so much a man a man a man to...
3,greedy,MACBETH:,MACBETH:\nI will not so be the sent of the con...
4,beam,ROMEO:,"ROMEO:\nWhat, thou shalt not to the country of..."
5,beam,JULIET:,JULIET:\nWhat should not be the common of the ...
6,beam,KING:,"KING:\nWhy, sir, thou shalt not to the commond..."
7,beam,MACBETH:,MACBETH:\nWhat is the common of the world of t...
8,topk_topp,ROMEO:,ROMEO:\nThe king that done of the more and she...
9,topk_topp,JULIET:,JULIET:\nHe will we are as the changed men to ...


# para sinusoidal

In [9]:
posenc_type_main = "sinusoidal"                     
checkpoint_main = "checkpoints/best_sinusoidal.pt"       

model_main = load_trained_model(posenc_type_main, checkpoint_main)

Modelo cargado (sinusoidal) desde checkpoints/best_sinusoidal.pt


C:\Users\USER\AppData\Local\Temp\ipykernel_11412\1931491145.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(checkpoint, map_location=device)


In [10]:
df_greedy = generate_samples(
    model_main,
    strategy="greedy",
    prompts=prompts,
    max_new_tokens=200,
)


,strategy,prompt,sample
0,greedy,ROMEO:,ROMEO:\nWhat the the the the the the the the t...
1,greedy,JULIET:,JULIET:\nWhat the the the the the the the the ...
2,greedy,KING:,KING:\nI I I I I I I I I I I mand the the the ...
3,greedy,MACBETH:,MACBETH:\nWhat the the the the the the the the...


In [11]:
df_beam = generate_samples(
    model_main,
    strategy="beam",
    prompts=prompts,
    max_new_tokens=200,
    beam_size=4,
    length_penalty=0.7,
)

,strategy,prompt,sample
0,beam,ROMEO:,ROMEO:\nWhat the the the the the the the the t...
1,beam,JULIET:,JULIET:\nWhat the the the the the the the the ...
2,beam,KING:,KING:\nWhat the the the the the the the the th...
3,beam,MACBETH:,MACBETH:\nWhat the the the that the the the th...


In [12]:
df_topp = generate_samples(
    model_main,
    strategy="topk_topp",
    prompts=prompts,
    max_new_tokens=200,
    top_k=50,
    top_p=0.9,
    temperature=0.8,
)

,strategy,prompt,sample
0,topk_topp,ROMEO:,"ROMEO:\nHat my come beser and ing, wit thint h..."
1,topk_topp,JULIET:,JULIET:\nHer you come for meer the the dot it\...
2,topk_topp,KING:,KING:\nMy man well that ind cof porie sto and ...
3,topk_topp,MACBETH:,MACBETH:\nAs the the courte the mere thers the...


# alibi

In [13]:
posenc_type_main = "alibi"                     
checkpoint_main = "checkpoints/best_alibi.pt"       

model_main = load_trained_model(posenc_type_main, checkpoint_main)

Modelo cargado (alibi) desde checkpoints/best_alibi.pt


C:\Users\USER\AppData\Local\Temp\ipykernel_11412\1931491145.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(checkpoint, map_location=device)


In [14]:
df_greedy = generate_samples(
    model_main,
    strategy="greedy",
    prompts=prompts,
    max_new_tokens=200,
)


,strategy,prompt,sample
0,greedy,ROMEO:,ROMEO:\nAnd the the the the the the the the th...
1,greedy,JULIET:,JULIET:\nThe the the the the the the the the t...
2,greedy,KING:,KING:\nWhe the the the the the the the the the...
3,greedy,MACBETH:,MACBETH:\nThe the the the the the the the the ...


In [15]:
df_beam = generate_samples(
    model_main,
    strategy="beam",
    prompts=prompts,
    max_new_tokens=200,
    beam_size=4,
    length_penalty=0.7,
)

,strategy,prompt,sample
0,beam,ROMEO:,ROMEO:\nWhe the the the the the the the the th...
1,beam,JULIET:,JULIET:\nThe the the the the the the the the t...
2,beam,KING:,KING:\nThe the the the the the the the the the...
3,beam,MACBETH:,MACBETH:\nThe the the the the the the the the ...


In [16]:
df_topp = generate_samples(
    model_main,
    strategy="topk_topp",
    prompts=prompts,
    max_new_tokens=200,
    top_k=50,
    top_p=0.9,
    temperature=0.8,
)

,strategy,prompt,sample
0,topk_topp,ROMEO:,"ROMEO:\nWe siters me hathe d ther boules,\nI t..."
1,topk_topp,JULIET:,JULIET:\nS:\nNous menoor f comor ste there wis...
2,topk_topp,KING:,"KING:\nWhay has nore chime certe, his d\nTheno..."
3,topk_topp,MACBETH:,MACBETH:\nAnd wa thay aroutou man t mean maith...
